# DSCI 100 Project Proposal

## Introduction

## Preliminary Exploratory Data Analysis

First we read the data from the repository and import libraries 

In [14]:
library(repr)
library(tidyverse) 
library(tidymodels)
options(repr.matrix.max.rows = 6) #limits output of dataframes to 6 rows

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.



In [6]:
# un-comment to download the data
# download.file(url = "https://github.com/Oscar-Yik/dsci-100-project/blob/3af4ff02686b68be2b21e76b3c5c68ec408c329f/data/heart_disease/processed.cleveland.data", destfile = "data/heart_disease/processed.cleveland.data")
# download.file(url = "https://github.com/Oscar-Yik/dsci-100-project/blob/3af4ff02686b68be2b21e76b3c5c68ec408c329f/data/heart_disease/processed.hungarian.data", destfile = "data/heart_disease/processed.hungarian.data")
# download.file(url = "https://github.com/Oscar-Yik/dsci-100-project/blob/3af4ff02686b68be2b21e76b3c5c68ec408c329f/data/heart_disease/processed.switzerland.data", destfile = "data/heart_disease/processed.switzerland.data")
# download.file(url = "https://github.com/Oscar-Yik/dsci-100-project/blob/3af4ff02686b68be2b21e76b3c5c68ec408c329f/data/heart_disease/processed.va.data", destfile = "data/heart_disease/processed.va.data")

In [5]:
heart_disease_cols <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")
heart_disease_C <- read_csv(file = "data/heart_disease/processed.cleveland.data", col_names = heart_disease_cols)
heart_disease_H <- read_csv(file = "data/heart_disease/processed.hungarian.data", col_names = heart_disease_cols)
heart_disease_S <- read_csv(file = "data/heart_disease/processed.switzerland.data", col_names = heart_disease_cols)
heart_disease_V <- read_csv(file = "data/heart_disease/processed.va.data", col_names = heart_disease_cols)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, chol, fbs, restecg, thalach, exang, slope, ca, thal
dbl (5): age, sex, cp, oldpeak, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal
dbl (5): age, sex, cp, chol, num

ℹ Use `spec()` to retr

We want the target column to be a factor type so the classfication algorithm can work properly 

In [12]:
heart_disease_C <- mutate(heart_disease_C, num = as_factor(num))
heart_disease_H <- mutate(heart_disease_H, num = as_factor(num))
heart_disease_S <- mutate(heart_disease_S, num = as_factor(num))
heart_disease_V <- mutate(heart_disease_V, num = as_factor(num))

We also want to get rid of missing values in the data so our models don't fail

In [24]:
# fill(heart_disease_C, "nocb")

With the prelimary data read and cleaned, we can split it into training and testing sets. For this project we will go with 30% training and 70% testing splits. 

In [18]:
heart_disease_C_split <- initial_split(heart_disease_C, prop = 0.3, strata = num)
heart_disease_C_train <- training(heart_disease_C_split)
heart_disease_C_test <- testing(heart_disease_C_split)

heart_disease_H_split <- initial_split(heart_disease_H, prop = 0.3, strata = num)
heart_disease_H_train <- training(heart_disease_H_split)
heart_disease_H_test <- testing(heart_disease_H_split)

heart_disease_S_split <- initial_split(heart_disease_S, prop = 0.3, strata = num)
heart_disease_S_train <- training(heart_disease_S_split)
heart_disease_S_test <- testing(heart_disease_S_split)

heart_disease_V_split <- initial_split(heart_disease_V, prop = 0.3, strata = num)
heart_disease_V_train <- training(heart_disease_V_split)
heart_disease_V_test <- testing(heart_disease_V_split)

In [19]:
glimpse(heart_disease_C_train)
glimpse(heart_disease_C_test)

Rows: 89
Columns: 14
$ age      <dbl> 41, 56, 52, 48, 49, 58, 60, 50, 66, 43, 44, 40, 71, 51, 54, 5…
$ sex      <dbl> 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1…
$ cp       <dbl> 2, 2, 3, 3, 2, 1, 4, 3, 1, 4, 3, 1, 2, 3, 3, 3, 2, 4, 3, 4, 3…
$ trestbps <dbl> 130, 120, 172, 130, 130, 150, 130, 120, 150, 150, 130, 140, 1…
$ chol     <dbl> 204, 236, 199, 275, 266, 283, 206, 219, 226, 247, 233, 199, 3…
$ fbs      <dbl> 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0…
$ restecg  <dbl> 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0…
$ thalach  <dbl> 172, 178, 162, 139, 171, 162, 132, 158, 114, 171, 179, 178, 1…
$ exang    <dbl> 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ oldpeak  <dbl> 1.4, 0.8, 0.5, 0.2, 0.6, 1.0, 2.4, 1.6, 2.6, 1.5, 0.4, 1.4, 0…
$ slope    <dbl> 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2…
$ ca       <chr> "0.0", "0.0", "0.0", "0.0", "0.0", "0.0", "2.0", "0.0", "0.0"…
$ thal     <chr> "3

## Methods

## Expected Outcomes and Significant